<a href="https://colab.research.google.com/github/suzannelittle/ca682i/blob/master/notebooks/solutions/2_3_7_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let's take another look at the data sampler set from Topic 1. I've made a sampler with a few sneaky errors and artefacts in it. We'll take a look at finding and fixing a few of them but which other ones can you find? 

In [ ]:
import pandas as pd

First we need to read the file in (`read_csv`), check that it looks okay (`head`) and check what the data types looks like (`info`)

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/suzannelittle/ca682i/master/data/sampler/DataSamplerMessy.csv")

Note that the url here is to a github repository and it's the direct link to the raw file. This lets us easily read the file in to the colab notebook without needing to upload the file. If you try this on your local computer you can either use the URL or download the .csv file and use a local filepath. 

In [ ]:
df.head()

Notice how the `read_csv` command has assumed that the first row is the column headers. No need to specify it this time!

In [ ]:
df.info()

Everything looks okay but some of the Dtypes don't match what we'd like to have and the index of the dataframe is just the row number. It would be much nicer to have the data indexed by the timestamp. Let's change the type of the "Time" column to datetime (`to_datetime`) and set that column to be the frame index.

In [ ]:
pd.to_datetime(df['Time']).head()   # Let's check that the conversion works first ...


Oh no! It's used American date formats! The date should be the 3rd of October not the 10th of March. This is a common data error. Let's fix this using the `dayfirst=True` argument. 

In [ ]:
pd.to_datetime(df['Time'], dayfirst=True).head()

That's better. Now let's change the column in the original dataframe by assigning the output from to_datetime to column 'Time' and check the data types again.

In [ ]:
df['Time'] = pd.to_datetime(df['Time'], dayfirst=True)
df.info()

See that column 'Time' is now datetime64 format that will let us apply [datetime specific functions](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html) to this column. I'm going to set the Time column to be the index of the dataframe.

In [ ]:
df = df.set_index('Time')
df.head()

A useful option now that we have a datetime index is we can get the values between 4:55pm and 5:05pm using the between_times() function. 

In [ ]:
df.between_time('16:55', '17:05')

# Starting to clean: Duplicates  

Now that we have the data set loaded let's try checking and cleaning.

Take a look at the documentation for [.duplicated](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.duplicated.html?highlight=duplicated#pandas.DataFrame.duplicated). Note that it returns a Series -- a list of Index : Boolean values where True shows the rows that are complete duplicates. You'll also need to check the Index for duplicates and to use the subset argument.

In [ ]:
df.duplicated()

To see the rows that are duplicated use the result from df.duplicated() as a mask for the dataframe. 

In [ ]:
df[df.duplicated()]

So the entry at 6:28pm is a completely duplicate row. Let's drop that.

In [ ]:
df_clean = df.drop_duplicates()

If you remember when we checked the info for the dataframe there were 152 entries. Now that we've dropped the one duplicate there should be 151. Let's check. We're also now working on a copy of the data (`df_clean`) since we are going to be making changes.

In [ ]:
df_clean.info()  # or you could use .shape

Using duplicated on the whole dataframe only checks for completely duplicated rows. Let's also check that the index is unique and there's a column titled 'ID' which sounds like it should be unique too. 

In [ ]:
df_clean[df_clean.index.duplicated()]

In [ ]:
df_clean[df_clean.duplicated(subset=['ID'], keep=False)]   # note the subset and the keep arguments

See how we can use the subet and keep arguments to get the rows that have a duplicate ID. The ID column is nonsense in this dataset but what might you do if you had to repair a duplicate ID? I'm going to change the second entry to SSCZ since I know that's not already in use here.

In [ ]:
df_clean.loc["2016-10-03 17:03:00", "ID"] = "SSCZ"

To change one instance in Pandas is oddly tricky. Using `.loc[row_index, column_name] = "new value"` will mostly work though it's not ideal and you may get a warning. In this case this is because we are working on a copy of df (`df_clean`) so the assignment of the new ID value will still work. Let's check.

In [ ]:
df_clean.loc["2016-10-03 17:03:00"]

In [ ]:
df_clean[df_clean.duplicated(subset=['ID'], keep=False)]

So the duplicate ID has been fixed and there are no other duplicate IDs in the sampler dataset. 

# Missing values?

In Pandas [missing values](https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html) are also referred to as NA. There are many options for handling these. First let's see if we have any in this dataset.


In [ ]:
df_clean[df_clean.ID.isna()]   # here I'm just checking the ID column for NaN

Looks like we have lost data that has been read and had the values recorded as NaN (missing value). For the moment we don't know how to fix this so let's go on and check the place names to see if there are are missing. It's dangerous to replace a fully missing row. In reality you'd probably need to go back to the data source and try to find the information. That is why Pandas will flag missing values with the special NaN/NaT entries. 

# Are the place names correct?

How can we check the content of the Ireland column for errors in place names? This can be a difficult one but luckily here we have a reference list! Use the dataset at https://github.com/suzannelittle/ca682i/blob/master/data/sampler/ireland_locations.xlsx?raw=true. Not that this is an Excel file (`read_excel`). 

You can then use the [.isin function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isin.html) to check if all the required locations in Ireland are correctly included in the dataset. This will return `True` for all values that are correct so use `~` to get the inverse values and show the entries that are not in the reference list. How many do you find?

In [ ]:
reference = pd.read_excel("https://github.com/suzannelittle/ca682i/blob/master/data/sampler/ireland_locations.xlsx?raw=true")

In [ ]:
reference.head()

In [ ]:
df_clean[~df_clean['Ireland'].isin(reference['Ireland'])]   # this shows the entries that are in df_clean but not in reference

In [ ]:
reference[~reference['Ireland'].isin(df_clean['Ireland'])]  # this shows the entries that are in reference but not in df_clean

# Whitespace

There's a lot of weird leading and trailing whitespace in the "Ireland" column! Use functions like  [`str.strip()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.strip.html), `str.lstrip()` and `str.rstrip()`. Note that here you want to include the str. as part of the function name when applying the function to the Series. 

In [ ]:
df_clean['Ireland'] = df_clean['Ireland'].str.strip()

# What about the Fruit column?

Take a look at the Fruit column and see if you can spot anything odd. Start by getting the unique values using the `unique()` function.

In [ ]:
df_clean['Fruit'].unique()

What errors or odd entries do you find? Can you use the `.loc[row_index, column_name] = value` method to fix some of them?

You might be interested to know that some entries were included for onion that should have been pear. How odd. You can replace all instances of a value using the `.replace('onion', 'pear')` function. Check the whitespace first!

In [ ]:
df_clean['Fruit'] = df_clean['Fruit'].str.strip()
df_clean['Fruit'] = df_clean['Fruit'].replace('onion', 'pear')
df_clean['Fruit'] = df_clean['Fruit'].replace('bannana', 'banana')

In [ ]:
df_clean['Fruit'].unique()

# Final thoughts

This notebook gives just a few examples of the types of functions you can use to explore and clean data using the Pandas library. There are similar functions in R. Compare these to the activities you did using Google Sheets and OpenRefine, which do you think are easier, more useful or more powerful? 